# MandiMetrics: Pakistan Food Price Intelligence

*   List item
*   List item


**Author:** Hottam Ud Din  
**Project Type:** End-to-End Data Science (Time Series & Geospatial Analysis)  
**Dataset Source:** [WFP Food Prices for Pakistan](https://data.humdata.org/dataset/wfp-food-prices-for-pakistan)

## 1. Project Overview
This project analyzes historical food price data (2004-2025) to understand the economic landscape of food security in Pakistan.
We aim to answer three key questions:
1. **Inflation:** Is the price hike due to currency devaluation or actual supply shortage?
2. **Disparity:** Which provinces bear the highest cost of living?
3. **Forecasting:** What will the price of essential staples be in the next 12 months across different regions?

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
import matplotlib.pyplot as plt

# Set plotting style
plt.style.use('fivethirtyeight')

print("Libraries Imported Successfully!")

Libraries Imported Successfully!


## 2. Data Loading and Preprocessing
The dataset contains a "meta-header" in the second row (index 0) which we need to remove. We also need to convert the `date` column to a DateTime object and ensure numeric columns are treated as numbers, not text.

In [2]:
# 1. Load the dataset
# Note: header=0 loads the first row as headers.
df = pd.read_csv("wfp_food_prices_pak.csv", header=0)

# 2. Cleaning
# The dataset often has a second row with tags like '#date', '#admin1'. We remove this.
df = df.drop(0)

# 3. Type Conversion
df['date'] = pd.to_datetime(df['date'])
numeric_cols = ['price', 'usdprice', 'latitude', 'longitude']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col])

# 4. Filter for Retail Prices
# We are interested in what the common man pays (Retail), not Wholesale.
df = df[df['pricetype'] == 'Retail']

# 5. Preview
print(f"Data Loaded: {df.shape[0]} rows, {df.shape[1]} columns")
df.head(3)

Data Loaded: 13420 rows, 16 columns


,date,admin1,admin2,market,market_id,latitude,longitude,category,commodity,commodity_id,unit,priceflag,pricetype,currency,price,usdprice
1,2004-01-15,BALOCHISTAN,Quetta,Quetta,295,30.19,67.01,cereals and tubers,"Rice (basmati, broken)",122,KG,actual,Retail,PKR,20.00,0.35
2,2004-01-15,BALOCHISTAN,Quetta,Quetta,295,30.19,67.01,cereals and tubers,Wheat flour,58,KG,actual,Retail,PKR,13.00,0.23
3,2004-01-15,KHYBER PAKHTUNKHWA,Peshawar,Peshawar,294,34.01,71.58,cereals and tubers,"Rice (basmati, broken)",122,KG,actual,Retail,PKR,17.59,0.31


## 3. Exploratory Data Analysis (EDA)
### 3.1 Inflation vs. Currency Devaluation
We compare the price of **Wheat Flour** in **PKR** (Local Currency) vs **USD**.
* **Hypothesis:** If the USD price is flat but PKR is rising, the issue is currency devaluation. If USD is also rising, it is a global supply shock.

In [3]:
# Select a staple commodity for analysis
target_commodity = "Wheat flour"
comm_df = df[df['commodity'] == target_commodity]

# Aggregate to National Average per month
national_trend = comm_df.groupby('date')[['price', 'usdprice']].mean().reset_index()

# Create Dual-Axis Plot
fig = go.Figure()

# Line 1: Price in PKR (Blue)
fig.add_trace(go.Scatter(
    x=national_trend['date'], y=national_trend['price'],
    name="Price (PKR)", line=dict(color='blue')
))

# Line 2: Price in USD (Red - Dotted)
fig.add_trace(go.Scatter(
    x=national_trend['date'], y=national_trend['usdprice'],
    name="Price (USD)", line=dict(color='red', dash='dot'),
    yaxis='y2' # Map to secondary Y-axis
))

# Layout updates
fig.update_layout(
    title=f"Inflation Analysis: {target_commodity} (PKR vs USD)",
    yaxis=dict(title="Price (PKR)"),
    yaxis2=dict(title="Price (USD)", overlaying='y', side='right'),
    template="plotly_white"
)

fig.show()

### 3.2 Provincial Price Disparity
We analyze the price distribution across provinces for the post-2020 period to understand current regional inequality.

In [4]:
# Filter for recent data (2020 onwards) to make the comparison relevant
recent_df = comm_df[comm_df['date'] > '2020-01-01']

# Box Plot
fig = px.box(recent_df, x='admin1', y='price', color='admin1',
             title=f"Regional Price Disparity (2020-Present): {target_commodity}",
             labels={'admin1': 'Province', 'price': 'Price (PKR)'})

fig.show()

## 4. Multi-Province Price Forecasting
We use **Facebook Prophet** to predict prices for the next 12 months.
We will train four separate models for **Punjab, Sindh, Balochistan, and Khyber Pakhtunkhwa** to compare their future inflation trajectories.

In [5]:
# 1. Define the 4 Major Provinces (Case Sensitive to match your data)
# Note: Based on your previous data screenshots, these are typically uppercase.
target_provinces = ['PUNJAB', 'SINDH', 'BALOCHISTAN', 'KHYBER PAKHTUNKHWA']
forecast_months = 12

# 2. Initialize the Plot
fig_forecast = go.Figure()

print(f"Starting Training for {target_commodity}...")

# 3. Loop through each province and train a dedicated AI model
for province in target_provinces:
    # Filter data for specific province and commodity
    prov_df = df[(df['commodity'] == target_commodity) & (df['admin1'] == province)]

    # Safety Check: Do we have enough data?
    if len(prov_df) < 50:
        print(f"Skipping {province}: Insufficient data points ({len(prov_df)}).")
        continue

    # Prepare Data for Prophet (Rename columns to 'ds' and 'y')
    prophet_df = prov_df.groupby('date')['price'].mean().reset_index()
    prophet_df = prophet_df.rename(columns={'date': 'ds', 'price': 'y'})

    # Train Model
    # yearly_seasonality=True helps capture harvest seasons
    m = Prophet(yearly_seasonality=True, daily_seasonality=False, weekly_seasonality=False)
    m.fit(prophet_df)

    # Create Future Dates
    future = m.make_future_dataframe(periods=forecast_months * 30)

    # Predict
    forecast = m.predict(future)

    # Add the "Trend Line" to our comparison chart
    fig_forecast.add_trace(go.Scatter(
        x=forecast['ds'],
        y=forecast['yhat'],
        name=f"{province} (Forecast)",
        mode='lines'
    ))
    print(f"✅ Model trained for {province}")

# 4. Finalize and Show the Chart
fig_forecast.update_layout(
    title=f"12-Month Price Forecast Comparison: {target_commodity}",
    xaxis_title="Date",
    yaxis_title="Predicted Price (PKR)",
    hovermode="x unified",
    template="plotly_white"
)

fig_forecast.show()

Starting Training for Wheat flour...


00:02:18 - cmdstanpy - INFO - Chain [1] start processing
00:02:19 - cmdstanpy - INFO - Chain [1] done processing


✅ Model trained for PUNJAB


00:02:19 - cmdstanpy - INFO - Chain [1] start processing
00:02:19 - cmdstanpy - INFO - Chain [1] done processing
00:02:20 - cmdstanpy - INFO - Chain [1] start processing


✅ Model trained for SINDH


00:02:20 - cmdstanpy - INFO - Chain [1] done processing
00:02:20 - cmdstanpy - INFO - Chain [1] start processing


✅ Model trained for BALOCHISTAN


00:02:20 - cmdstanpy - INFO - Chain [1] done processing


✅ Model trained for KHYBER PAKHTUNKHWA


## 5. Conclusion
* **Inflation:** The analysis confirms that recent price hikes are driven significantly by currency devaluation, as the USD price has remained relatively stable compared to the PKR price.
* **Seasonality:** The model successfully identified price drops during harvest seasons (typically Q2).
* **Regional Disparity:** Foretasted trends indicate that **Balochistan** continues to face higher prices due to logistics, while **Punjab** remains the most affordable region for Wheat Flour.